## Crawling Naver Stock Repots

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
# 1. URL
url = 'https://finance.naver.com/research/company_list.naver?page=1'

In [15]:
# 2. request(URL) > response(HTML)
response = requests.get(url)
response

<Response [200]>

In [21]:
response.text[:200]

'<!--  global include -->\n\n\t\n\t\n\t\n\t\n\t\n<html lang=\'ko\'>\n<head>\n\n\n\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t<title>종목분석 리포트 : 네이버페이 증권</title>\n\t\t\t\n\t\t\n\t\n\n\n\n\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\n'

In [5]:
# 3. HTML > BeautifulSoup > css-selector > DataFrame

In [27]:
dom = BeautifulSoup(response.content, 'html.parser')
type(dom) # select(css-selector), select_one()

bs4.BeautifulSoup

In [57]:
selector = 'table.type_1 > tr'
elements = dom.select(selector)
len(elements)

49

In [99]:
elements[1]

<tr><td class="blank_07" colspan="6"></td></tr>

In [71]:
elements[2]

<tr>
<td style="padding-left:10">
<a class="stock_item" href="/item/main.naver?code=066570" title="LG전자">LG전자</a>
</td>
<td><a href="company_read.naver?nid=77126&amp;page=1">AI데이터센터 냉각 시장 공략</a><img alt="NEW" class="ico_new" height="8" src="https://ssl.pstatic.net/imgstock/images5/ico_research_new.gif" width="8"/></td>
<td>교보증권</td>
<td class="file"><a href="https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf" target="_blank"><img align="absmiddle" alt="pdf" src="https://ssl.pstatic.net/imgstock/images5/down.gif"/></a></td>
<td class="date" style="padding-left:5px">24.09.23</td>
<td class="date">219</td>
</tr>

In [67]:
element = elements[2]
tag = element.select('td')
len(tag), tag

(6,
 [<td style="padding-left:10">
  <a class="stock_item" href="/item/main.naver?code=066570" title="LG전자">LG전자</a>
  </td>,
  <td><a href="company_read.naver?nid=77126&amp;page=1">AI데이터센터 냉각 시장 공략</a><img alt="NEW" class="ico_new" height="8" src="https://ssl.pstatic.net/imgstock/images5/ico_research_new.gif" width="8"/></td>,
  <td>교보증권</td>,
  <td class="file"><a href="https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf" target="_blank"><img align="absmiddle" alt="pdf" src="https://ssl.pstatic.net/imgstock/images5/down.gif"/></a></td>,
  <td class="date" style="padding-left:5px">24.09.23</td>,
  <td class="date">219</td>])

In [95]:
data = {}
data['stock_name'] = tag[0].select_one('a').text
data['stock_link'] = tag[0].select_one('a').get('href')
data['title'] = tag[1].select_one('a').text
data['title_link'] = tag[1].select_one('a').get('href')
data['writer'] = tag[2].text
data['pdf_link'] = tag[3].select_one('a').get('href')
data['date'] = tag[4].text
data['pv'] = tag[5].text
data

{'stock_name': 'LG전자',
 'stock_link': '/item/main.naver?code=066570',
 'title': 'AI데이터센터 냉각 시장 공략',
 'title_link': 'company_read.naver?nid=77126&page=1',
 'writer': '교보증권',
 'pdf_link': 'https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf',
 'date': '24.09.23',
 'pv': '219'}

In [ ]:
selector = 'table.type_1 > tr'
elements = dom.select(selector)

In [125]:
# enumerate
rows = []
for idx, element in enumerate(elements):
    tag = element.select('td')
    # print(idx, len(tag))
    if len(tag) == 6:
        data = {}
        data['stock_name'] = tag[0].select_one('a').text
        data['stock_link'] = tag[0].select_one('a').get('href')
        data['title'] = tag[1].select_one('a').text
        data['title_link'] = tag[1].select_one('a').get('href')
        data['writer'] = tag[2].text
        data['pdf_link'] = tag[3].select_one('a').get('href')
        data['date'] = tag[4].text
        data['pv'] = tag[5].text
        rows.append(data)

In [115]:
df = pd.DataFrame(rows)
df.tail(2)

,stock_name,stock_link,title,title_link,writer,pdf_link,date,pv
28,동국S&C,/item/main.naver?code=100130,금리 하락으로 미국에서 훈풍이 불어온다,company_read.naver?nid=77098&page=1,iM증권,https://stock.pstatic.net/stock-research/compa...,24.09.19,1501
29,SOOP,/item/main.naver?code=067160,"경쟁사 스트리머 이적, 트래픽 유입 기대",company_read.naver?nid=77097&page=1,신한투자증권,https://stock.pstatic.net/stock-research/compa...,24.09.19,1455


In [133]:
# enumerate
data = list('ABCD')
print(data)
count = 0
for row in data:
    print(count, row)
    count += 1

['A', 'B', 'C', 'D']
0 A
1 B
2 C
3 D


In [141]:
print(list(enumerate(data)))
for idx, row in enumerate(data):
    print(idx, row)

[(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D')]
0 A
1 B
2 C
3 D


In [147]:
df.tail(1)

,stock_name,stock_link,title,title_link,writer,pdf_link,date,pv
29,SOOP,/item/main.naver?code=067160,"경쟁사 스트리머 이적, 트래픽 유입 기대",company_read.naver?nid=77097&page=1,신한투자증권,https://stock.pstatic.net/stock-research/compa...,24.09.19,1455


In [ ]:
# file download

In [153]:
# os package : 파일 시스템을 관리하는 파이썬 패키지
import os
# 파일목록출력
os.listdir()[-2:]

['07_naver_relational_keywords.ipynb', '08_naver_stock_report.ipynb']

In [157]:
path = 'reports'
# 디렉토리, 파일 존재 여부 확인
os.path.exists(path)

False

In [159]:
if not os.path.exists(path): # 디렉토리 없으면 True
    os.makedirs(path) # 디렉토리생성

In [161]:
os.path.exists(path)

True

In [165]:
os.listdir()[-2:]

['08_naver_stock_report.ipynb', 'reports']

In [181]:
# 1. url
title = df.loc[0, 'title']
pdf_link = df.loc[0, 'pdf_link']
title, pdf_link

('AI데이터센터 냉각 시장 공략',
 'https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf')

In [187]:
# 2. request(pdf_link) > response(pdf)
response = requests.get(pdf_link)
response

<Response [200]>

In [200]:
# 3. pdf > save(/reports)
filename = f'{path}/{title}.pdf'
print(filename)
with open(filename, 'wb') as file:
    file.write(response.content)

reports/AI데이터센터 냉각 시장 공략.pdf


In [ ]:
# pickle : RAM > SSD : SSD > RAM

In [204]:
os.listdir('reports')

['AI데이터센터 냉각 시장 공략.pdf']

In [214]:
os.makedirs(path)

In [216]:
import shutil
shutil.rmtree(path)
# os.path.exists(path)
os.makedirs(path)

In [218]:
os.listdir(path)

[]

In [224]:
for idx, row in df.iterrows():
    # print(idx, row['title'], row['pdf_link'])
    print(idx, end=' ')
    title, pdf_link = row['title'], row['pdf_link']
    response = requests.get(pdf_link)
    filename = f'{path}/{title}.pdf'
    with open(filename, 'wb') as file:
        file.write(response.content)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

In [226]:
os.listdir(path)

['2024년 3분기 부진 터널 통과해야!.pdf',
 '3Q24 Preview',
 '3Q24 일시적인 실적 둔화,  분위기 반전이 필...pdf',
 '3분기는 비수기, 그러나   주주 가치 환원에 ...pdf',
 'AI데이터센터 냉각 시장 공략.pdf',
 'Metsera, 너는 계획이 다 있구나.pdf',
 'Moment of Truth.pdf',
 'Re-rating 구간 돌입.pdf',
 '가치가 높아지는 F박스와 K패션 해외 진출 수...pdf',
 '경쟁력 있는 CDMO, 여기에도 있다.pdf',
 '경쟁사 스트리머 이적, 트래픽 유입 기대.pdf',
 '경증 아토피 치료의 글로벌  시장 판도를 바...pdf',
 '금리 하락으로 미국에서 훈풍이 불어온다.pdf',
 '길어지고 있는 기다림.pdf',
 '높아지는 Peak sales.pdf',
 '롯데렌탈의 쏘카 지분 추가취득 당분간 중단.pdf',
 '미국 타워 판가도 인상.pdf',
 '빅파마들의 RPT 방향, 우리도 간다.pdf',
 '설계와 시공능력으로 액침냉각 사업 확대.pdf',
 '속도가 느려도, 방향성은 맞다.pdf',
 '액침냉각, 기술적 강점을 확보해 나가는 중.pdf',
 '엔지니어링 플라스틱 소재 개발 및 고도화로 ...pdf',
 '엘사.pdf',
 '의대 열풍, 나만 믿어.pdf',
 '이튼의 견고한 성장 파트너.pdf',
 '지속가능한 성장.pdf',
 '코스닥 방사성의약품 기업으로 진입 .pdf',
 '콥데이 후기',
 '확대되는 TROP2 ADC 치료제 시장.pdf',
 '환율 모멘텀 발생 예상. 자사주 추가 매입 가...pdf']